In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama



In [10]:
llm = ChatOllama(model="llama3.2:1b")

prompt_template = PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}? Return the name of the capital only.",
)

output_parser = StrOutputParser()
answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))
print(answer)

Paris


In [12]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""
    Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    IMPORTANT: Return the information in JSON format.
    IMPORTANT: Any string except the JSON should not be returned.
    """,
    input_variables=["country"],
)

output_parser = JsonOutputParser()
answer = output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country": "France"})))

print(answer)

OutputParserException: Invalid json output: {
  "capital": "Paris",
  "population": 67_204_000,
  "language": "French",
  "currency": "Euro"
}

In [18]:
# 픽스
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

structured_llm = llm.with_structured_output(CountryDetail)
answer = structured_llm.invoke(country_detail_prompt.invoke({"country": "France"}))
print(answer)



capital='Paris' population=65276857 language='French' currency='Euro'
